# Code for generating and running PH of noisy networks - dev

In [ ]:
# Need to load in all the network generation functions here?



In [1]:
# Load packages

using Pkg
using Statistics
using Plots
using LinearAlgebra
using Distances
using Eirene
using StatsBase
using Random
using Distributions
using JLD

function is_symmetric(adj)
    tf = isequal(adj,transpose(adj))
    if !tf
        println("Matrix is not symmetric!")
    end
    return tf
end


function make_iid_weighted_graph(nNodes)
    
    adj = rand(nNodes,nNodes)
    for i = 1:nNodes, j = 1:nNodes
        # symmetrize
        adj[j,i] = adj[i,j]
    end
    
    # Set diagonal to 0
    adj[diagind(adj)] .= 0
    
    # Check for symmetry
    tf = is_symmetric(adj)
    
    # Return adjacency matrix
    return adj
end
    

function make_random_geometric(nNodes,dims)
    
    # Generate random coordinates in [0,1)^dims
    randomCoordinates = rand(dims,nNodes)
    
    # Compute pairwise distances between nodes (points in [0,1)^dims)
    adj = pairwise(Euclidean(), randomCoordinates, dims = 2)
    adj = 1 ./adj
    adj[diagind(adj)] .= 0
    
    # Check for symmetry
    tf = is_symmetric(adj)
    
    return adj
end


function check_density(G)
    
    # Determine number of nodes and edges
    nNodes = size(G,1)
    nPosEdges = length(G[G.> 0])./2    # Divide by 2 because this counts upper and lower triangular edges
    

    # Calculate the edge density as the number of nonzero edges over the total possible
    edge_density = nPosEdges./binomial(nNodes,2)
    
    return edge_density
end


function threshold_graph(G,rho,nNodes)
    
    # Count edges and edges we need to keep
    nEdges = binomial(nNodes,2)
    thresh_edge_number = ceil(Int,rho*nEdges)
    
    # Obtain value on which to threshold
    sorted_edges = sort(unique([G...]),rev = true)
    thresh_edge_value = sorted_edges[thresh_edge_number]
    
    # Make copy and threshold G
    G_thresh = deepcopy(G)
    G_thresh[G_thresh.< thresh_edge_value] .= 0
    
    # Test graph density
    rho_test = check_density(G_thresh)
    #println("Graph edge density is $rho_test")
    
    return G_thresh
end


function betticurveFromBarcode(barcode,nSteps)
    # nSteps will be nEdges here

    betti_curve = zeros(1,nSteps)   # Column-major order

    for bar in eachrow(barcode)
    
        bar_first = Int(bar[1])
        bar_last = Int(bar[2]) 
    
        # Add to betti curve
        betti_curve[1,bar_first:bar_last] = betti_curve[1,bar_first:bar_last] .+1 
    end
    
    return betti_curve
end

function make_ring_lattice_wei(nNodes)
    # Create weighted ring lattice
    
    if !iseven(nNodes)
        v = ones(1,(nNodes-1)) ./ [transpose(collect(1:floor(nNodes/2))) transpose(reverse(collect(1:floor(nNodes/2))))]
        v = [0 v]
        adj = deepcopy(v)
        for n in 1:(nNodes-1)
            adj = [adj; zeros(1,n) transpose(v[1,1:(nNodes-n)])]
        end
        adj = adj .+ (1/(nNodes^4))*rand(nNodes,nNodes)
        adj = adj+transpose(adj)
        adj[diagind(adj)].=0
        
        
    end
    
    return adj
end


function make_cosine_geometric(nNodes,dims)
    
    # Generate random coordinates in [0,1)^dims
    randomCoordinates = rand(dims,nNodes)
    
    # Compute pairwise distances between nodes (points in [0,1)^dims)
    adj = pairwise(CosineDist(), randomCoordinates, dims = 2)
    
    # Check for symmetry
    tf = is_symmetric(adj)
    
    return adj
end




function make_dot_product(nNodes,dims)
    # Generate random coordinates in [0,1)^dims
    randomCoordinates = rand(dims,nNodes)
    
    # Compute pairwise distances between nodes (points in [0,1)^dims)
    adj = randomCoordinates' * randomCoordinates
    
    # Check for symmetry
    tf = is_symmetric(adj)
    
    return adj
end


function make_dumb_2modular(nNodes,p_in,p_out)
    
    adj = zeros(nNodes,nNodes)
    
    community_assignments = [ones(1,Int(nNodes/2)) 2*ones(1,Int(nNodes/2))]
    
    for n1 in 1:nNodes
        for n2 in n1:nNodes
            
            if isequal(community_assignments[n1],community_assignments[n2])
                
                we = 0
                r = rand(1,1)[1]
                
                while r < p_in
                    we = we+1
                    r = rand(1,1)[1]
                end
                
                adj[n1,n2] = we + 0.0001*rand(1,1)[1]
                adj[n2,n1] = adj[n1,n2]
                
            else
                
                we = 0
                r = rand(1,1)[1]
                
                while r < p_out
                    we = we+1
                    r = rand(1,1)[1]
                end
                
                adj[n1,n2] = we + 0.0001*rand(1,1)[1]
                adj[n2,n1] = adj[n1,n2]
            end
            
        end
    end
    
     # Check for symmetry
    tf = is_symmetric(adj)
    
    return adj
end

   


    
function make_dumb_4modular(nNodes,p_in,p_out)
    
    adj = zeros(nNodes,nNodes)
    
    community_assignments = [ones(1,Int(nNodes/4)) 2*ones(1,Int(nNodes/4)) 3*ones(1,Int(nNodes/4)) 4*ones(1,Int(nNodes/4))]
    
    for n1 in 1:nNodes
        for n2 in n1:nNodes
            
            if isequal(community_assignments[n1],community_assignments[n2])
                
                we = 0
                r = rand(1,1)[1]
                
                while r < p_in
                    we = we+1
                    r = rand(1,1)[1]
                end
                
                adj[n1,n2] = we + 0.0001*rand(1,1)[1]
                adj[n2,n1] = adj[n1,n2]
                
            else
                
                we = 0
                r = rand(1,1)[1]
                
                while r < p_out
                    we = we+1
                    r = rand(1,1)[1]
                end
                
                adj[n1,n2] = we + 0.0001*rand(1,1)[1]
                adj[n2,n1] = adj[n1,n2]
            end
            
        end
    end
    
    # Check for symmetry
    tf = is_symmetric(adj)
    
    return adj
end




function make_dev_DiscreteUniform_configuration_model(nNodes,a,b)
    
    # Create a configuration model using the discrete uniform distribution between a and b.
    
    # Define distribution
    d = DiscreteUniform(a,b)
    strength_sequence = rand(d,nNodes)

    stubs = deepcopy(strength_sequence)
    adj = zeros(nNodes,nNodes)

    nodes_left = []
    # While stubs are left
    while sum(stubs)>0

        # Find which nodes have stubs left
        nodes_left = findall(stubs.>0)

        # If only one node is left, we did badly
        if length(nodes_left) == 1
            println("One node left - try again")
            
            # Currently this is a draft so we will allow it.
            break
        end


        # nodes_left contains cartesian indices. Can access them with nodes_left[i][j]. Not anymore
        node1,node2 = sample(nodes_left,2, replace = false)

        # Add edge to adjacency matrix
        adj[node1, node2] = adj[node1, node2] + 1 

        # Update stubs
        stubs[node1] = stubs[node1] - 1
        stubs[node2] = stubs[node2] - 1

        
    end

    # Now we only added edges to one side of the adjacency matrix.
    adj = adj+adj'
    
    # Add noise
    adj = adj .+ (1/(nNodes^4))*rand(nNodes,nNodes)
    adj = adj+transpose(adj)
    adj[diagind(adj)].=0
    
    # Check for symmetry
    tf = is_symmetric(adj)
    
    return adj
end





function make_dev_Geometric_configuration_model(nNodes,p,scale_factor)
    
    # Create a configuration model using the Geometric distribution with parameter p. To get enough edges,
    # scale by scale_factor
    
    # Define distribution
    d = Geometric(p)
    strength_sequence = scale_factor*rand(d,nNodes)

    stubs = deepcopy(strength_sequence)
    adj = zeros(nNodes,nNodes)

    nodes_left = []
    # While stubs are left
    while sum(stubs)>0

        # Find which nodes have stubs left
        nodes_left = findall(stubs.>0)

        # If only one node is left, we did badly
        if length(nodes_left) == 1
            println("One node left - try again")
            
            # Currently this is a draft so we will allow it.
            break
        end


        # nodes_left contains cartesian indices. Can access them with nodes_left[i][j]. Not anymore
        node1,node2 = sample(nodes_left,2, replace = false)

        # Add edge to adjacency matrix
        adj[node1, node2] = adj[node1, node2] + 1 

        # Update stubs
        stubs[node1] = stubs[node1] - 1
        stubs[node2] = stubs[node2] - 1

        
    end

    # Now we only added edges to one side of the adjacency matrix.
    adj = adj+adj'
    
    # Add noise
    adj = adj .+ (1/(nNodes^4))*rand(nNodes,nNodes)
    adj = adj+transpose(adj)
    adj[diagind(adj)].=0
    
    # Check for symmetry
    tf = is_symmetric(adj)
    
    return adj
end







┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1242
┌ Info: Precompiling Distances [b4f34e82-e78d-54a5-968a-f98e89d6e8f7]
└ @ Base loading.jl:1242
┌ Info: Precompiling Eirene [9c0f25c4-2ca1-5870-89f6-52640788da1d]
└ @ Base loading.jl:1242


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15395272323990132915\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8064446779989406228\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15395272323990132915\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1242


make_dev_Geometric_configuration_model (generic function with 1 method)

## Choose weighed network and generate replicates

--------------------------------
## CHANGE PARAMETERS HERE
---------------------------------

In [3]:
## Pick from one of the graph models and generate out to THRESH edge density

nReps = 50
rho = 0.5      # Threshold at edge density = rho
nNodes = 80
save_figures = 1  # Boolean to save figure pdfs
save_data = 0    # Boolean to save data  

# for geometricConf
p = 0.01
scale_factor = 100

# for RG
dims = 3

# for discreteUniformConf
a = 0
b = 1000

# for cosineFeometric
dims = 3


# Graph model name one of "geometricConf", "IID" , "RG", "discreteUniformConf", "cosineGeometric", "RL", "SWBM"
graph_model_name = "geometricConf"

"geometricConf"

In [3]:


weighted_graph_array = zeros(nNodes,nNodes,nReps)
weighted_graph_array_ord = zeros(nNodes,nNodes,nReps)
betti_file_name = []
parameters = []
for rep in 1:nReps
    
    
    if graph_model_name == "geometricConf"
        G_i = make_dev_Geometric_configuration_model(nNodes,p,scale_factor)
        parameters = [nReps, rho, nNodes, p, scale_factor]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == " IID"
        G_i = make_iid_weighted_graph(nNodes)
        parameters = [nReps, rho, nNodes]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "RG"
        G_i = make_random_geometric(nNodes,dims)
        parameters = [nReps, rho, nNodes, dims]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
       
        
    elseif graph_model_name == "discreteUniformConf"
        G_i = make_dev_DiscreteUniform_configuration_model(nNodes,a,b)
        parameters = [nReps, rho, nNodes, a, b]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "cosineGeometric"
        G_i = make_cosine_geometric(nNodes,dims)
        parameters = [nReps, rho, nNodes, dims]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "RL"
        G_i = make_ring_lattice_wei(nNodes)
        parameters = [nReps, rho, nNodes]
        betti_file_name = "$(graph_model_name)"
        
        for parameter in parameters
            betti_file_name = "$(betti_file_name)_$(replace(string(parameter),"." => ""))"
        end
        
        
    elseif graph_model_name == "SWBM"
        println("load data you fool")
    end
    
        

    
    # Threshold at rho
    G_i_thresh = threshold_graph(G_i,rho,nNodes)
    
    weighted_graph_array[:,:,rep] = G_i_thresh
    
    
    edge_list_ranks = denserank([G_i...], rev = true)   # so highest edge weight gets assigned 1
    G_i_ord = reshape(edge_list_ranks,(nNodes,nNodes))
    G_i_ord[diagind(G_i_ord)] .= 0
    weighted_graph_array_ord[:,:,rep] = G_i_ord
    
    
 
    
end
println("Naming files $(betti_file_name)")

# We can add noise to the entire array at the same time

weighted_graph_array_iidNoise = deepcopy(weighted_graph_array)
weighted_graph_array_iidNoise_ord = zeros(nNodes,nNodes,nReps)
for rep in 1:nReps
    
    G_i = weighted_graph_array_iidNoise[:,:,rep]
    G_i[G_i.>0] .= G_i[G_i .>0 ] .+1
    
    # Now the real values are all >1, so we can add noise which will be < 1
    
    G_rand = make_iid_weighted_graph(nNodes)
    G_i_rand = deepcopy(G_i)
    G_i_rand[G_i_rand .== 0] .= G_rand[G_i_rand .== 0]
    
    # So anything <1 in G_i_rand will be from random noise, and any entry >1 will be real matrix
    
    weighted_graph_array_iidNoise[:,:,rep] = G_i_rand
    
    # Now create the ranked matrix for running eirene -- it will save us many headaches later.
    edge_list_ranks = denserank([G_i_rand...], rev = true)   # so highest edge weight gets assigned 1
    G_i_rand_ord = reshape(edge_list_ranks,(nNodes,nNodes))
    G_i_rand_ord[diagind(G_i_rand_ord)] .= 0
    weighted_graph_array_iidNoise_ord[:,:,rep] = G_i_rand_ord
    
    
end



    
# Plot an example weighted graph and after we add noise

gr()

test_mat = 19
p0a = heatmap(weighted_graph_array[:,:,test_mat], aspect_ratio = :equal)
p0b = heatmap(weighted_graph_array_iidNoise[:,:,test_mat], aspect_ratio = :equal)
p0c = heatmap(weighted_graph_array_iidNoise_ord[:,:,test_mat], aspect_ratio = :equal)

plot(p0a,p0b,p0c,layout = (1,3))


if save_figures == 1
    savefig("../Figures/$(betti_file_name).pdf")
end

if save_data == 1
    save("../processed_data/$(betti_file_name)_graphs.jld",
     "weighted_graph_array", weighted_graph_array,
     "weighted_graph_array_iidNoise", weighted_graph_array_iidNoise,
     "parameters", parameters)
end



One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node left - try again
One node lef

## Running Eirene

In [4]:
# Loop through replicates and run eirene

maxDim = 3
nEdges = binomial(nNodes,2)
barcode_array = Array{Array{Float64}}(undef,nReps,3)
betticurve_array = zeros(nEdges,maxDim,nReps)
beginTime = time()

C = 4

for rep = 1:nReps
    
    G_i = weighted_graph_array_iidNoise_ord[:,:,rep]
    
    C = Eirene.eirene(G_i,model = "vr", maxdim = maxDim, record = "none")
    println("finished running eirene")
    for k in collect(1:maxDim)
        
        barcode_array[rep,k] = barcode(C,dim=k)
        betti_curve_i = betticurveFromBarcode(barcode_array[rep,k],nEdges)
        betticurve_array[:,k,rep] = betti_curve_i[1,:]
        
    end 
   
    if rep%10 == 0
        printstyled("\nRun $(rep) finished.\n",color = :cyan)
    end

    C
end



endTime = time() - beginTime
printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

if save_data == 1
    save("../processed_data/$(betti_file_name)_eirene_output.jld",
        "barcode_array", barcode_array,
        "betticurve_array", betticurve_array)
end


printstyled("\nDone :)", color = :blue)


finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene

Run 10 finished.
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene

Run 20 finished.
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene

Run 30 finished.
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished r

In [5]:
## Plot average betti curves

betticurve_avg = dropdims(mean(betticurve_array,dims=3), dims = 3)
betticurve_std = dropdims(std(betticurve_array,dims=3), dims = 3)

x_axis = collect(1:nEdges) ./nEdges
p1a = plot(x_axis,betticurve_avg[:,1],linewidth = 1.5, ribbon = betticurve_std[:,1], label = "Betti_1")
plot!(x_axis,betticurve_avg[:,2],linewidth = 1.5, ribbon = betticurve_std[:,2], label = "Betti_2")
plot!(x_axis,betticurve_avg[:,3],linewidth = 1.5, ribbon = betticurve_std[:,3], label = "Betti_3")
title!("$(betti_file_name)")
ylabel!("Betti_k")
xlabel!("Edge Density")

plot(p1a,layout = (1,1))

if save_figures == 1
    savefig("../Figures/$(betti_file_name).pdf")
end


In [ ]:
using LightGraphs
using PyCall

nx = pyimport("networkx")

In [ ]:
gmn = "test"
a = 2
b = 3
c = [2,3,4]
d = "$(gmn)_$(a)_$(b)_$(c...)"
e = 5
f = "$(d)_$(e)"


In [ ]:
macro get_var_name(arg)
   string(arg)
end

variablex  = 6

a = @Name(variablex)

In [ ]:
replace("4.32", "." => "")